# Примеры реализованных на SQL запросов 

In [ ]:
SELECT title
FROM stackoverflow.posts
WHERE title IS NOT NULL and user_id in(
select user_id
from 
(SELECT distinct user_id,
    COUNT(id) over (partition by user_id) as badge
FROM stackoverflow.badges) as all_badges
where badge > 1000
)

In [ ]:
SELECT ROUND(AVG(post_count), 0)
FROM
(
    SELECT p.creation_date:: date,
           COUNT(p.id) as post_count
    FROM stackoverflow.posts p
    JOIN stackoverflow.post_types pt ON p.post_type_id = pt.id
    WHERE pt.type = 'Question' AND p.creation_date:: date BETWEEN '01.11.2008' AND '18.11.2008'
    GROUP BY creation_date:: date
) count_question;

In [ ]:
with t1 AS (
    select id,
           views,
           CASE
               when views < 100 then 3
               when views < 350 then 2
               when views >= 350 then 1
           END AS group_n
    from stackoverflow.users
    where location like '%United States%' AND views != 0), 

    t2 AS (
    SELECT *,
           MAX(views) OVER (PARTITION BY group_n) AS max_views
    FROM t1
    ORDER BY views DESC
        )
SELECT id,
       group_n,
       views
FROM t2
WHERE views = max_views
ORDER BY views DESC, id

In [ ]:
WITH u AS (
    SELECT id as user_id,
           creation_date:: date
    FROM stackoverflow.users
    WHERE creation_date:: date BETWEEN '01.11.2008' AND '30.11.2008'
       ),
       
u_count AS (
    SELECT creation_date, 
           EXTRACT(DAY FROM creation_date) AS day,
           COUNT(user_id) AS user_count
    FROM u
    GROUP BY creation_date
    ORDER BY creation_date
)

SELECT day,
       user_count,
       SUM(user_count) OVER(ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) AS accumulation_sum
FROM u_count

In [ ]:
WITH t1 AS (
    SELECT user_id,
       COUNT(DISTINCT id) AS posts_count
FROM stackoverflow.posts
GROUP BY user_id
ORDER BY posts_count desc
)
   
SELECT  DISTINCT EXTRACT (WEEK FROM creation_date:: date),
        MAX(creation_date) OVER(PARTITION BY EXTRACT (WEEK FROM creation_date:: date))
FROM stackoverflow.posts
WHERE user_id = (SELECT user_id
                 FROM t1
                 WHERE posts_count = (SELECT MAX(posts_count)
                                      FROM t1))
      AND DATE_TRUNC('MONTH', creation_date):: date = '01.10.2008'